![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 2 - Clasificación de género de películas

El propósito de este proyecto es que puedan poner en práctica, en sus respectivos grupos de trabajo, sus conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 2: Clasificación de género de películas"

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 8. Sin embargo, es importante que avancen en la semana 7 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 8, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/2c54d005f76747fe83f77fbf8b3ec232).

## Datos para la predicción de género en películas

![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/moviegenre.png)

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros.

Agradecemos al profesor Fabio González, Ph.D. y a su alumno John Arevalo por proporcionar este conjunto de datos. Ver https://arxiv.org/abs/1702.01992

## Ejemplo predicción conjunto de test para envío a Kaggle
En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [5]:
# Visualización datos de test
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


# Preprocesamiento de datos

## CountVectorizer vs TFIDVectorizer

In [17]:
#vect = CountVectorizer(max_features=1000) 
#vect = TfidfVectorizer(max_features=10000) #Mejoró resultados

# Definición de variables predictoras (X)
#X_dtm = vect.fit_transform(dataTraining['plot'])
#X_dtm.shape

Los resultados mejoraron usando TFIDVectorizer en lugar de CountVectorizer. Esto es porque mientras que CountVectorizer simplemente cuenta el número de veces que aparece una palabra en un texto, TF-IDF no solamente cuenta el número de veces sino que también evalúa que tan importante es la palabra en ese texto. Esto se hace gracias a la penalización de algunas palabras que el algoritmo considera menos importantes. Precisamente este factor es el que puede darle ventaja de TF-IDF sobre CountVectorizer, dado que este último pesa todas las palabras por igual, mientras que el primero ayuda a lidiar con las palabras más repetitivas y las penaliza.

##  TFIDVectorizer + Stop words 

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Cargamos los stopwords en una variable
stop_words_ = list(set(stopwords.words('english')))

#vect = TfidfVectorizer(max_features=10000,stop_words=stop_words_) #Quitando Stop Words Mejoró los modelos

# Definición de variables predictoras (X)
#X_dtm = vect.fit_transform(dataTraining['plot'])
#X_dtm.shape

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joseh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


En segundo lugar, probamos el eliminar los stopwords o palabras más comunes usando como tokenizador el TFID vectorizer el cual nos da mejores resultados. En este caso, la combinación de la penalización de las palabras más comunes por parte del TFID y la eliminación de las palabras más repetitivas aumentó el valor predictivo del modelo.


## TFIDVectorizer + Stop words + Lematización de verbos

In [16]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Creamos un objeto WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Definición de la función que tenga como parámetro texto y devuelva una lista de lemas
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

## Función para lematizar los verbos en cada comentario - empeoró los resultados
#def lemmatize_verbs(text):
    #tagged_text = nltk.pos_tag(text.split()) # Part of speech tagging
    #lemmatized_tokens = [lemmatizer.lemmatize(word, 'v') if tag.startswith('V') else word for word, tag in tagged_text] # Lematización solo de verbos
    #lemmatized_text = ' '.join(lemmatized_tokens)
    #return lemmatized_text

# Vectorizamos los comentarios 
vect = TfidfVectorizer(max_features=10000,stop_words=stop_words_,analyzer=split_into_lemmas)

# Definición de variables predictoras (X)
X_dtm = vect.fit_transform(dataTraining['plot'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joseh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\joseh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Por último, usamos lematización, proceso en el cual cada palabra se busca en un diccionario el cual ha sido previamente calculado y este, para cada palabra, nos va a decir cuál debe ser su representación. Usamos dos tipos de lematización, una que da la definición de la función para que tenga como parámetro texto y devuelva una lista de lemas, y el otro para lematizar los verbos en cada comentario. Los mejores valores se dieron con el primero. 

Por lo tanto, usamos el TFID y el Stop words dado que nos han dado los mejores valores. En conclusión, usando la mayoría de técnicas de preprocesamiento, los mejores valores los encontramoso usando FIDVectorizer + Stop words + Lematización de verbos.

In [10]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])
y_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

In [11]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [12]:
# Definición y entrenamiento
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=10, n_jobs=-1,
                                                     random_state=42))

In [13]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.8174317146859066

In [14]:
# transformación variables predictoras X del conjunto de test
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = clf.predict_proba(X_test_dtm)

In [15]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_RF.csv', index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.133458,0.116152,0.022739,0.045186,0.407097,0.156613,0.036416,0.508193,0.067529,0.109457,...,0.044859,0.085238,0.000033,0.320989,0.065122,0.037139,0.040952,0.256173,0.029684,0.019143
4,0.160392,0.102109,0.022760,0.051285,0.342970,0.228589,0.087583,0.528298,0.071921,0.081535,...,0.025551,0.073773,0.000896,0.164395,0.064094,0.007363,0.021283,0.223114,0.032375,0.019143
5,0.148692,0.113762,0.022964,0.037688,0.278543,0.388135,0.021350,0.541159,0.067870,0.076762,...,0.025551,0.179523,0.000000,0.275629,0.067862,0.007363,0.023352,0.373886,0.036102,0.019143
6,0.169550,0.114147,0.022964,0.035555,0.324615,0.166085,0.036024,0.576920,0.067184,0.073459,...,0.025555,0.090484,0.000000,0.297274,0.100241,0.007363,0.023298,0.287376,0.069545,0.019143
7,0.195160,0.125724,0.023151,0.036100,0.314815,0.249551,0.036499,0.449110,0.076839,0.082914,...,0.025271,0.170956,0.000023,0.209046,0.230252,0.007363,0.021034,0.269066,0.039313,0.019143
